### Autofisher

Requires:
* RuneLite

Limitations:
* Barbarian Fishing

Setup:
* Open Bag
* Drop Chat Window
* Zoom to 50% of max
* Point Camera
* Set items you dont want to drop

In [1]:
#User Settings
#Type in the game window name
window_name = "RuneLite - Lord Jakarn"
#Items to drop as a python list i.e. [1, 2, 3, 15]
drop = list(range(4, 29)[:])

### Initialization
* Import/Install Dependencies
* Set Game Window Size
* Move Game Window to Specific Location

In [2]:
import pyautogui as ag
import cv2 as cv
import numpy as np
import pygetwindow as pgw
from matplotlib import pyplot as plt
from pynput.mouse import Button, Controller as MouseController
from pynput.keyboard import Key, Controller as KeyboardController
import time
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

#set up game screen
game_window = pgw.getWindowsWithTitle(window_name)[0]
game_window.moveTo(0,0)
game_window.resizeTo(1000,800)

#store item co-ordinates
co_ords = {1 : [792, 515], 2 : [832, 513], 3 : [879, 514], 4 : [920, 515],
           5 : [793, 554], 6 : [837, 550], 7 : [876, 551], 8 : [921, 550],
           9 : [792, 587], 10 : [837, 587], 11 : [879, 589], 12 : [922, 587],
           13 : [792, 624], 14 : [834, 623], 15 : [877, 624], 16 : [920, 625],
           17 : [794, 658], 18 : [835, 660], 19 : [877, 659], 20 : [920, 659],
           21 : [794, 694], 22 : [835, 696], 23 : [878, 694], 24 : [919, 696],
           25 : [792, 733], 26 : [837, 731], 27 : [879, 732], 28 : [922, 732]}

#load images
images = {"bag_full" : None, "bag_empty" : None, "fishing" : None, "not_fishing" : None,
         "status" : None, "no_status" : None, "fish1" : None, "fish2" : None, "fish3" : None}
for i, v in enumerate(images):
    image = cv.imread(v + ".jpg")
    image = cv.cvtColor(np.array(image), cv.COLOR_BGR2GRAY)
    images[v] = image

In [5]:
#screengrab - returns screenshot of windowed area
def screen_grab():
    image = ag.screenshot()
    image = cv.cvtColor(np.array(image), cv.COLOR_BGR2GRAY)
    x, y, h, w = 0, 0, 800, 1000
    image = image[y:y+h, x:x+w]
    return image

In [6]:
#reads interaction text
def interact_text_read(image):
    image = image
    x, y, h, w = 5, 27, 20, 70
    crop = image[y:y+h, x:x+w]
    text = pytesseract.image_to_string(crop)
    if text[0:7] == "Use-rod":
        found = True
    else:
        found = False
    return found

In [7]:
def drop_items():
    MouseController().position = [400, 10]
    time.sleep(0.5)
    MouseController().click(Button.left, 1)
    time.sleep(0.5)
    for i in drop:
        with KeyboardController().pressed(Key.shift):
            time.sleep(1)
            xy = co_ords[i]
            MouseController().position = xy
            time.sleep(0.2)
            MouseController().click(Button.left, 1)
            time.sleep(0.5)

In [8]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

In [9]:
fishing = False

def fishing_status():
    x, y, h, w = 10, 50, 35, 90
    image = screen_grab()
    image = image[y:y+h, x:x+w]
    if mse(image, images["fishing"]) > mse(image, images["not_fishing"]):
        fishing = False
    else:
        fishing = True
    return fishing

In [10]:
full = False

def bag_full():
    x, y, h, w = 50, 650, 110, 400
    image = screen_grab()
    image = image[y:y+h, x:x+w]
    if mse(image, images["status"]) > mse(image, images["no_status"]):
        full = False
    else:
        full = True
    return full

In [11]:
fish2 = images['fish2']

def fish():
    while interact_text_read(screen_grab()) == False:
        time.sleep(0.5)
        x, y = img_find()
        MouseController().position = [x, y]
        time.sleep(0.5)
    MouseController().click(Button.left, 1)
    time.sleep(10)
    return None

In [12]:
import imutils

def img_find():
    # load the image image, convert it to grayscale, and detect edges
    template = cv.Canny(fish2, 50, 200)
    (tH, tW) = template.shape[:2]
    #cv.imshow("Template", template)

    image = screen_grab()
    found = None
    for scale in np.linspace(0.2, 1.0, 20)[::-1]:
        resized = imutils.resize(image, width = int(image.shape[1] * scale))
        r = image.shape[1] / float(resized.shape[1])
        # if the resized image is smaller than the template, then break
        # from the loop
        if resized.shape[0] < tH or resized.shape[1] < tW:
            break
        # detect edges in the resized, grayscale image and apply template
        # matching to find the template in the image
        edged = cv.Canny(resized, 50, 200)
        result = cv.matchTemplate(edged, template, cv.TM_CCOEFF)
        (_, maxVal, _, maxLoc) = cv.minMaxLoc(result)
        # draw a bounding box around the detected region
        clone = np.dstack([edged, edged, edged])
        cv.rectangle(clone, (maxLoc[0], maxLoc[1]),
            (maxLoc[0] + tW, maxLoc[1] + tH), (0, 0, 255), 2)
        # if we have found a new maximum correlation value, then update
        # the bookkeeping variable
        if found is None or maxVal > found[0]:
            found = (maxVal, maxLoc, r)
    # unpack the bookkeeping variable and compute the (x, y) coordinates
    # of the bounding box based on the resized ratio
    (_, maxLoc, r) = found
    (startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
    (endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))
    # draw a bounding box around the detected result and display the image
    cv.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
    x = startX + ((endX - startX)/2)
    y = startY + ((endY - startY)/2)
    return x, y

x, y = img_find()
print(x, y)

565.5 525.5


In [13]:
while True == True:
    time.sleep(1)
    print("fishing status:", fishing_status())
    if fishing_status() == False:
        print("bag_status:", bag_full())
        if bag_full() == True:
            print("dropping")
            drop_items()
        else:
            print("fishing")
            fish()

fishing status: False
bag_status: False
fishing
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: False
bag_status: True
dropping
fishing status: False
bag_status: False
fishing
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: True
fishing status: 

KeyboardInterrupt: 